*Note: Place this notebook in STHN directory.*

# TGM (Satellite to Thermal)

In [ ]:
%pip install PyYAML transformers timm h5py torch torchvision faiss-cpu einops opencv-python

## Image

[my_eval_pix2pix.py](./global_pipeline/my_eval_pix2pix.py)
- [Input Image](./global_pipeline/my_eval_pix2pix.py:69)
- [Output Image](./global_pipeline/my_eval_pix2pix.py:94)


In [ ]:
!py ./global_pipeline/my_eval_pix2pix.py --resume="./maps/models/TGM_nocontrast/best_model.pth" --dataset_name=none --datasets_folder ./maps --G_net unet --GAN_upsample bilinear --GAN_resize 1024 1024

## Folder

[my_TGM_folder2folder.py](./global_pipeline/my_TGM_folder2folder.py)
- [Input Folder](./global_pipeline/my_TGM_folder2folder.py:296)
- [Output Folder](./global_pipeline/my_TGM_folder2folder.py:297)

In [ ]:
!py ./global_pipeline/my_TGM_folder2folder.py --resume="./maps/models/TGM_nocontrast/best_model.pth" --dataset_name=none --datasets_folder ./maps --G_net unet --GAN_upsample bilinear --GAN_resize 1024 1024

# SHN (Matching)

In [ ]:
%pip install kornia scikit-image wandb openpyxl

[my_myevaluate.py](./local_pipeline/my_myevaluate.py)
- [Input Image](./local_pipeline/my_myevaluate.py:118)
- [Output Image](./local_pipeline/my_myevaluate.py:119)
- [Output Excel](./local_pipeline/my_myevaluate.py:168)

## One-Stage

In [4]:
!python3  ./local_pipeline/my_myevaluate.py --dataset_name none --eval_model js_models/1536_one_stage/STHN.pth --val_positive_dist_threshold 512 --lev0 --database_size 1536 --corr_level 4 --test

/home/rpl/Map Matching/STHN-JetsonONX8/local_pipeline/model/network.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
torch.Size([1, 256, 64, 64]) torch.Size([1, 256, 64, 64])
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
❌ Error in image 0: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "/opt/pytorch/c10/cuda/CUDACachingAllocator.cpp":1131, please report a bug to PyTorch. 
torch.Size([1, 256, 64, 64]) torch.Size([1, 256, 64, 64])
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
NvMapMemAllocInternalTagged: 1075072515 error 12
NvMapMemHandleAlloc: error 0
NvMapMemAlloc

## Two-Stages

In [ ]:
!python3 ./local_pipeline/my_myevaluate.py --dataset_name none --eval_model js_models/1536_two_stages/STHN.pth --val_positive_dist_threshold 512 --lev0 --database_size 1536 --corr_level 4 --test